# Uma breve introdução a Python para Análises Hidrológicas
<br>
<img style="float: left; padding-right: 15px; padding-left: 0px;" src="source/logo_flood_proofs.png" width="260px" align=”left” >

<div style="text-align: justify">Este é um Jupyter Notebook, um ambiente de desenvolvimento interactivo baseado na web que permite criar e partilhar códigos Python.
Primeiro, o que é **Python***? Python é uma linguagem de programação de alto nível e de uso geral. Pode ser utilizado para escrever software numa grande variedade de domínios de aplicação, incluindo a hidrologia. Python pode ser utilizado para efectuar cálculos numéricos, análises estatísticas ou para aceder e plotar dados (mesmo grandes conjuntos de dados). <br>
No caderno de notas Jupyter, a *Python shell* está incorporada. A **shell** é onde se pode escrever e executar uma linha (ou múltiplas linhas) de código.
Python é de código aberto, e estão disponíveis vários pacotes que cobrem muitos campos científicos e tecnológicos.

Vamos começar a utilizar Python como **calculador***:

In [ ]:
190/3


Se necessário, podemos atribuir este resultado a uma **variável**, e utilizar a variável para cálculos futuros ou para outras operações (como a conversão para o inteiro e a visualização do resultado).

In [ ]:
result = 190/3
new_result = result - 14
int_result = int(new_result)
print(int(int_result))


E se quisermos realizar alguns cálculos mais complexos? Podemos importar o pacote **math**, carregando várias funções matemáticas (tais como a raiz quadrada)

In [ ]:
import math
sqrt_result = math.sqrt(int_result)
print(sqrt_result)


E se quisermos trabalhar não com um único valor, mas com um **vector** composto por múltiplos valores?

In [ ]:
import numpy as np
array = [1,4,100,3,-2]
print(array)
print('------> complete array maximum: ' + str(np.max(array)))
array_nan = array
array_nan[2] = np.nan
print(array_nan)
print('------> incomplete array maximum: ' + str(np.max(array)))


Será o último resultado correcto? Não deveria ser 4 o novo valor máximo? Podemos utilizar uma função específica para contabilizar os "Nan" ou os valores em falta: *np.nanmax* (parte de pacote numpy)

In [ ]:
print('------> incomplete array maximum: ' + str(np.nanmax(array)))


### Tratando de Séries Temporais (timeseries)
Podemos gerar **séries temporais** (múltiplos valores com data associada) e exibi-la em um gráfico? Claro que sim!
Comecemos por importar algumas bibliotecas úteis!

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
import wget
import os

# We can also define personal function, this one for example allow to visualize the output of dropdowns menus
def on_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            print("Selected value: " + change['new'])


Podemos agora proceder com uma série cronológica aleatória!

In [ ]:
ts = pd.Series(np.random.randn(365), index=pd.date_range('1/1/2010', periods=365))
plt.figure()
ts.plot(style='b-', label='Random timeseries')
plt.legend()


O mais interessante é trabalhar com as séries temporais existentes. A NOAA fornece um conjunto de dados global de valores diários de precipitação e temperatura, o **Global Historical Climate Network Daily** (https://www.ncdc.noaa.gov/ghcn-daily-description).
O conjunto de dados pode ser consultado por país seleccionando-o a partir do seguinte menu descendente.

In [ ]:
# Load list of available countries and generate dropdown selector
with open('source/ghcnd-countries.txt', 'r') as file:
    list_country = [line for line in file]
country_chooser = widgets.Dropdown(
    options=['Choose a country'] + list_country,
    value='Choose a country',
    description='Country:',
    disabled=False,
)
country_chooser.observe(on_change)
display(country_chooser)


Ao executar a próxima peça de código será mostrado um menu descendente da estação disponível para o país seleccionado. Por favor, seleccione a estação de interesse.
**NB!** Alguns países (Brasil, Austrália e EUA) têm demasiadas estações e podem quebrar o sistema, para esses países a selecção manual da estação é viável!

In [ ]:
# Generate the list of the available stations
country_code = country_chooser.value[0:2]
list_stations = pd.read_fwf('source/ghcnd-stations.txt',
                            widths=[2,9,9,10,7,4,31,3,10],
                            header=None, usecols=[0,1,2,3,4,6], 
                            names=['COUNTRY','CODE','LAT','LON','ELEV','NAME'])
list_stations_in = list_stations.loc[list_stations['COUNTRY']==country_code].sort_values('NAME', ascending=True)

if len(list_stations_in)<4000:
    station_chooser = widgets.Dropdown(
        options=['Choose a station'] + list(list_stations_in['COUNTRY'] + list_stations_in['CODE'] + ' ' + list_stations_in['NAME']),
        value='Choose a station',
        description='Station:',
        disabled=False,
    )
    station_chooser.observe(on_change)
    display(station_chooser)
else:
    print('Station list is too long! Please, manually choose the station code from the available list at the web address https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt among the ones with first column starting with ' + country_code)


A próxima peça de código descarregará a série a partir da série NOAA e analisará as variáveis disponíveis. 
Escolher a variável para a análise a partir do menu descendente:

In [ ]:
# Insert information only for manually selecting the station code (for Brazil, Austalia and US)
section_code =  None     # es: 'BR00B7-0400'
section_name = None      # es: 'SAO JOAO DE IRACEMA'
###############################################################################################

if section_code is None:
    section_code = station_chooser.value.split(' ', 1)[0]
    section_name = station_chooser.value.split(' ', 1)[1]
file_name = section_code +'.csv'
out_path = 'meteo/' + file_name

# Check if the file has been already downloaded
if os.path.isfile(out_path):
    print('Section ' + section_code + ' ' + section_name + ' already downloaded!')
    print('DONE!')
else:
    print('Dowloading section ' + section_code + ' ' + section_name + '... It can take some times!')
    https_address = 'https://www.ncei.noaa.gov/data/global-historical-climatology-network-daily/access/'
    wget.download(https_address + file_name, out = out_path)
    print('DONE!')
    
# Open the file and analyse the available variables
info_station = pd.read_csv(out_path, header=0, usecols=['STATION','NAME','LATITUDE','LONGITUDE','ELEVATION'], nrows=1)
full_series = pd.read_csv(out_path, header=0, index_col='DATE', parse_dates=True, usecols=lambda c: c in {'DATE','PRCP','SNWD','TMAX','TMIN','TAVG'}) #, usecols=[1,2,3], names=['date','type','val'])
dic_vars={'precipitation':['PRCP', 'Rainfall(mm)'], 'temperature mean':['TAVG','Temperature(°C)'], 'temperature max':['TMAX','Temperature(°C)'], 'temperature min':['TMIN','Temperature(°C)'], 'snow depth':['SNWD', 'Snow depth(cm)']}

available_vars = [i for i in dic_vars if dic_vars[i][0] in full_series.columns]

var_chooser = widgets.Dropdown(
    options=['Choose a variable'] + available_vars,
    value='Choose a variable',
    description='Vars available:',
    disabled=False,
)

var_chooser.observe(on_change)
display(info_station.style.hide_index())
display(var_chooser)


Agora podemos representar interactivamente uma das séries de tempos disponíveis, escolhendo a estação, a variável e também os limites de tempo: 

In [ ]:
# Insert information only for choosing a sub-period of the whole series
time_start = None    # Set a date in the format 'YYYY-MM-DD' or None for plot the series from the beginning
time_end = None      # Set a date in the format 'YYYY-MM-DD' or None for plot the series up to the end
####################################################################

# Read data series
variable = var_chooser.value
temp_series = full_series[[dic_vars[variable][0]]]/10

# Set time range
time_start = temp_series.first_valid_index() if time_start is None else pd.to_datetime(time_start,format='%Y-%m-%d')
time_end = temp_series.last_valid_index() if time_end is None else pd.to_datetime(time_end,format='%Y-%m-%d')
if time_start > time_end:
    raise ValueError("time_start is larger than time_end, verify your data!")
time_range = pd.date_range(time_start,time_end,freq='1D')
temp_series = temp_series.reindex(time_range)

display(temp_series)

# Manage plot
ax = temp_series.plot(style='b', title=variable + ' at ' + section_name, figsize=(15,5))
ax.set_xlabel("")
ax.set_ylabel(dic_vars[variable][1])
ax.get_legend().remove()
plt.show()


As séries cronológicas podem ser facilmente geridas com Python para operações estatísticas e de resampling:

In [ ]:
# The resample frequency can be set, e.g., to annual 'Y' or monthly 'M'
temp_resampled_max = temp_series.reindex(time_range).resample('M').max()
temp_resampled_min = temp_series.reindex(time_range).resample('M').min()
temp_resampled_avg = temp_series.reindex(time_range).resample('M').mean()

# Manage plot
ax = temp_resampled_max.plot(style='r', title=variable + ' at ' + section_name, figsize=(15,5))
temp_resampled_min.plot(style='b',ax=ax)
temp_resampled_avg.plot(style='g',ax=ax)
ax.set_xlabel("")
ax.set_ylabel(dic_vars[variable][1])
plt.legend(['max','min','avg'])
plt.savefig(section_name + '_' + variable + '.png')
plt.show()


## Flood frequency analysis using Python
Podemos utilizar a Python para calcular as estatísticas de cheias numa série temporais de caudal. Referência a *hydro-informatics.github.io*<br>

A ocorrência de eventos (extremos) de inundação relevantes pode ser expressa como **período de retorno**, expressando o intervalo médio de recorrência de um evento de uma certa magnitude em unidades de tempo. É o inverso da **probabilidade de superação** (a probabilidade de um evento de certa magnitude ou superior).<br>
Um pressuposto significativo no cálculo do período de retorno é que os eventos individuais são considerados independentes. Isto significa que, para um determinado ano, a probabilidade de ocorrência de uma inundação de 100 anos é de 1/100.
Aqui abaixo uma tabela que mostra os intervalos de recorrência e as probabilidades de ocorrências relacionadas.

| Período de retorno (anos) | Probabilidade anual de superação (%) |
| --- | --- |
| 2 | 50 |
| 5 | 10 |
| 10 | 10 |
| 50 | 2 |
| 100 | 1 |
| 500 | 0.2 |

No início devemos importar algumas bibliotecas úteis:

In [ ]:
import numpy as np
import pandas as pd
import glob
import wget
import os
import zipfile


Então, devemos importar os **dados de caudal**. 
Vamos ver o que os ficheiros da série "txt" podem ser encontrados dentro da pasta "discharge".
Outros dados podem ser descarregados a partir do portal de dados GRDC através da realização de um pedido personalizado (https://portal.grdc.bafg.de/applications/public.html?publicuser=PublicUser#dataDownload/Stations). O pedido será avaliado pelo fornecedor de dados e dentro de pouco tempo será fornecido um e-mail com uma url de descarregamento. 
A url pode ser inserida e analisada directamente com esta ferramenta:

In [ ]:
## Modify this section by inserting the download address provided by the GRDC website for download more data
# es: download_link = 'https://portal.grdc.bafg.de/grdcdownload/external/53c13313-e359-4f61-bb9d-d803b2ab74e1/2021-07-01_16-52.zip'
download_link = None
############################################################################################################

if not download_link is None:
    os.makedirs('temp', exist_ok=True)
    wget.download(download_link, out= 'temp/')
    with zipfile.ZipFile('temp/' + os.path.basename(download_link), 'r') as zip_ref:
        zip_ref.extractall('discharge/')
    
files = glob.glob("discharge/*.txt")
print(files)


A lista não nos diz muito sobre o conteúdo do ficheiro, podemos abrir um destes ficheiros para compreender o conteúdo de cada ficheiro 

**NOTE! A numeração Python começa a partir de 0!!**:

In [ ]:
# Read a preview of the file
number_of_lines = 40

with open(files[0],'rb') as file:
    for i in np.arange(0,number_of_lines,1): 
        line = file.readline().decode('ISO-8859-1')
        print(str(i) + ' ' + line)


As linhas entre 8 e 18 de cada ficheiro contêm toda a informação sobre a estação, podemos usar a capacidade python de gerir diferentes tipos de ficheiro para resumir essas informações numa tabela:

In [ ]:
# Read the 11 lines after line 8 (Python numbering starts from 0!)
for ind, file in enumerate(files,0):
    data = pd.read_csv(file, skiprows=8, nrows=11, sep=":", encoding='ISO-8859-1', header=None, names=['cod','val'])    
    if ind == 0:
        list_vars = [i.replace('# ','') for i in data['cod']]
        df_stations = pd.DataFrame(index=np.arange(0,len(files),1),columns=list_vars)
    data['cod'] = list_vars
    data = data.set_index(['cod'])
    for var in list_vars:
        df_stations.loc[ind][var] = data.loc[var].values[0].strip()

df_stations = df_stations.set_index(["GRDC-No."])
display(df_stations)


Podemos agora escolher que estação a analisar, fornecendo o seu código para identificar o ficheiro relacionado, vamos começar, por exemplo, com o **AWASH WENZ em MELKA KUNTIRE** (COD: 1577100):

In [ ]:
# Please, specify an available station code
station_code = '1577100'
##############################################################################################

# Read data from line 37
df = pd.read_csv("discharge/" + station_code + "_Q_Day.Cmd.txt",
                 header=None,
                 sep=";",
                 skiprows=37,
                 names=["Date", "Time","Q"],
                 parse_dates=[0],
                 index_col=["Date"])
df['Q']=df['Q'].astype(float)
ax = df.plot(title=df_stations.loc[station_code]["River"] + " at " + df_stations.loc[station_code]["Station"], figsize=(15,5))
ax.set_ylabel('Q (m3/s)')


Existem valores nulos na série, que correspondem a valores nulos, podemos geri-los repalhando com "np.nan", que é o valor nulo standard do numpy:

In [ ]:
# Replace negative values with "null"
df.loc[df['Q']<0,'Q']=np.nan
ax = df.plot(title=df_stations.loc[station_code]["River"] + " at " + df_stations.loc[station_code]["Station"], figsize=(15,5))
ax.set_ylabel('Q (m3/s)')


Esta é a série cronológica completa: temos de seleccionar apenas os **máximos anuais**. É bastante simples com *pandas dataframe*, podemos fazer uma nova amostragem do nosso conjunto de dados (que foi indexado com datas).

In [ ]:
# Resample using the annual maximum value
df_ymax = df.resample("Y").max()
df_ymax["year"] = df_ymax.index.year
df_ymax.reset_index(inplace=True, drop=True)
df_ymax = df_ymax.dropna()
print(df_ymax)

# Manage plot
df_ymax.plot(kind='scatter',x='year',y='Q',color='red', figsize=(15,5))
plt.show()


### Análise do período de retorno
Devemos calcular a probabilidade de excedência *Pr*, e o intervalo de recorrência resultante.
Pr is defined as: $Pr_{i} = \frac{(n-i+1)}{n+1}$\
Onde *n* é o número total de anos de observação e *i* é a posição do evento.

In [ ]:
# Sort in increasing order
df_ymax_sorted = df_ymax.sort_values(by="Q")
n = df_ymax_sorted.shape[0]
df_ymax_sorted.insert(0, "rank", range(1, 1 + n))
print(df_ymax_sorted)


A **probabilidade de superação** ( *pr* ) pode ser calculada aplicando a fórmula:

In [ ]:
df_ymax_sorted["pr"] = (n - df_ymax_sorted["rank"] + 1) / (n + 1)
print(df_ymax_sorted.tail())


O **intervalo de recorrência** ( *período de retorno* ) é o inverso da probabilidade, portanto:

In [ ]:
df_ymax_sorted["return-period"] = 1 / df_ymax_sorted["pr"]
print(df_ymax_sorted.tail())


Uma vez criada a tabela (*dataframe*) com toda a informação necessária (**Probabilidade** e **Período de retorno**) podemos visualizá-la para mostrar o intervalo de recorrência de cada caudal observado.  Vale a pena mencionar que esta análise e o gráfico resultante se referem apenas aos valores observados.<br>
Para extrapolar o intervalo de recorrência para além do período de observação (os valores de inundação de 1 em 100 anos, por exemplo) é necessário um modelo de previsão (Gumbel, GEV, etc.).

In [ ]:
df_ymax_sorted.plot.scatter(y="Q",
                         x="return-period",
                         title="Return period [years] ",
                         color='blue',
                         grid=True,
                         fontsize=14,
                         logy=False,
                         label="Sorted values",
                         figsize=(15,10))


# Isto é para esta breve introdução prática a Python! 

## Pode exercitar-se um pouco:
 
### Exercício 1

Escolher a estação meteorológica de Tirana na Albânia:
* qual é o valor máximo de temperatura alcançado na série temporais? (a partir do gráfico, lembre-se que também pode adicionar linhas de código ao bloco de notas jupyter se quiser. Ajuda: use a função np.max)

Escolha outra estação meteorológica noutro país à sua escolha:
* guarde o gráfico de chuva. Carregar a png resultante na plataforma Moodle.

### Exercício 2

Escolher a estação hidrológica no rio Buzi (código do *dataframe*):

* qual é o valor de caudal para o Período de Retorno = 5?
* qual é o valor de caudal com uma probabilidade de superação = 0.5?

Escolha outra estação hidrológica em Moçambique:

* pode traçar os períodos de retorno numa escala logarítmica (para y)? Guarde o gráfico. Carregar a png resultante na plataforma Moodle.